In [33]:
# My pc couldnot handle it so I used Kaggle kernel, and this notebook spent 50 mins running on Kaggle Kernel
# The accuarcy: 26.14%

In [34]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [35]:
train_df = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip', parse_dates=['Dates'])
test_df = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip', parse_dates=['Dates'])

In [36]:
train_df.sample(5)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
45212,2014-10-04 15:19:00,ASSAULT,INFLICT INJURY ON COHABITEE,Saturday,SOUTHERN,"ARREST, BOOKED",8TH ST / MISSION ST,-122.413161,37.777457
528060,2007-11-29 07:40:00,ASSAULT,ATTEMPTED SIMPLE ASSAULT,Thursday,MISSION,NONE,4600 Block of 19TH ST,-122.440944,37.759071
456700,2008-11-30 17:15:00,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Sunday,TARAVAL,NONE,700 Block of RANDOLPH ST,-122.469357,37.714196
135081,2013-07-27 17:00:00,NON-CRIMINAL,LOST PROPERTY,Saturday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421
456892,2008-11-30 01:50:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Sunday,SOUTHERN,NONE,500 Block of HOWARD ST,-122.397032,37.787825


In [37]:
test_df.sample(1)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
842037,2003-06-25 18:00:00,VEHICLE THEFT,STOLEN TRUCK,Wednesday,BAYVIEW,NONE,0 Block of MISSISSIPPI ST,-122.39496,37.765906


In [38]:
print('Number of Categories: ', train_df.Category.nunique())
print('Number of PdDistricts: ', train_df.PdDistrict.nunique())
print('Number of DayOfWeeks: ', train_df.DayOfWeek.nunique())
print('_________________________________________________')
# Show some useful Information
train_df.info()

Number of Categories:  39
Number of PdDistricts:  10
Number of DayOfWeeks:  7
_________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
Dates         878049 non-null datetime64[ns]
Category      878049 non-null object
Descript      878049 non-null object
DayOfWeek     878049 non-null object
PdDistrict    878049 non-null object
Resolution    878049 non-null object
Address       878049 non-null object
X             878049 non-null float64
Y             878049 non-null float64
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 60.3+ MB


In [39]:
train_df = train_df.drop('Resolution', axis=1)
train_df.sample(1)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y
234433,2012-03-22 11:00:00,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Thursday,PARK,1200 Block of STANYAN ST,-122.451687,37.760484


In [40]:
train_df.drop_duplicates(inplace=True)
train_df.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
test_df.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

In [41]:
train_df['Date'] = pd.to_datetime(train_df.Dates)
test_df['Date'] = pd.to_datetime(test_df.Dates)

train_df = train_df.drop('Dates', axis=1)
test_df = test_df.drop('Dates', axis=1)
train_df.sample(1)

,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y,Date
475426,ASSAULT,THREATS AGAINST LIFE,Wednesday,SOUTHERN,200 Block of 9TH ST,-122.412845,37.774817,2008-09-03 13:30:00


In [42]:
# Confirm that it was parsed to Datetime
train_df.Date.dtype

dtype('<M8[ns]')

In [43]:
train_df['IsDay'] = 0
train_df.loc[ (train_df.Date.dt.hour > 6) & (train_df.Date.dt.hour < 20), 'IsDay' ] = 1
test_df['IsDay'] = 0
test_df.loc[ (test_df.Date.dt.hour > 6) & (test_df.Date.dt.hour < 20), 'IsDay' ] = 1

train_df.sample(3)

,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y,Date,IsDay
120107,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Friday,INGLESIDE,100 Block of PEABODY ST,-122.407315,37.710356,2013-10-04 09:00:00,1
270606,OTHER OFFENSES,FALSE PERSONATION TO RECEIVE MONEY OR PROPERTY,Friday,NORTHERN,2300 Block of SACRAMENTO ST,-122.431566,37.790092,2011-09-09 22:00:00,0
267656,ASSAULT,ATTEMPTED SIMPLE ASSAULT,Saturday,TARAVAL,3300 Block of RIVERA ST,-122.502220,37.745676,2011-09-24 13:25:00,1


In [44]:
days_to_int_dic = {
        'Monday': 1,
        'Tuesday': 2,
        'Wednesday': 3,
        'Thursday': 4,
        'Friday': 5,
        'Saturday': 6,
        'Sunday': 7,
}
train_df['DayOfWeek'] = train_df['DayOfWeek'].map(days_to_int_dic)
test_df ['DayOfWeek'] = test_df ['DayOfWeek'].map(days_to_int_dic)

train_df.DayOfWeek.unique()

array([3, 2, 1, 7, 6, 5, 4])

In [45]:
train_df['Hour'] = train_df.Date.dt.hour
train_df['Month'] = train_df.Date.dt.month
train_df['Year'] = train_df.Date.dt.year
train_df['Year'] = train_df['Year'] - 2000 # The Algorithm doesn't know the difference. It's just easier to work like that

test_df['Hour'] = test_df.Date.dt.hour
test_df['Month'] = test_df.Date.dt.month
test_df['Year'] = test_df.Date.dt.year
test_df['Year'] = test_df['Year'] - 2000 # The Algorithm doesn't know the difference. It's just easier to work like that

train_df.sample(1)

,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y,Date,IsDay,Hour,Month,Year
704088,ASSAULT,INFLICT INJURY ON COHABITEE,5,BAYVIEW,0 Block of DOUBLEROCK ST,-122.387695,37.720555,2005-05-20 16:44:00,1,16,5,5


In [46]:
train_df = pd.get_dummies(train_df, columns=['PdDistrict'])
test_df  = pd.get_dummies(test_df,  columns=['PdDistrict'])
train_df.sample(2)

,Category,Descript,DayOfWeek,Address,X,Y,Date,IsDay,Hour,Month,...,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
353600,PROSTITUTION,LOITERING FOR PURPOSE OF PROSTITUTION,3,17TH ST / SHOTWELL ST,-122.416230,37.763634,2010-06-16 20:00:00,0,20,6,...,0,0,0,1,0,0,0,0,0,0
479637,NON-CRIMINAL,LOST PROPERTY,6,POWELL ST / MARKET ST,-122.407546,37.784401,2008-08-09 19:30:00,1,19,8,...,0,0,0,0,0,0,0,1,0,0


In [47]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['CategoryInt'] = le.fit_transform(train_df.Category)
train_df.sample(5)
# cat_le.classes_

,Category,Descript,DayOfWeek,Address,X,Y,Date,IsDay,Hour,Month,...,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,CategoryInt
94397,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,6,100 Block of JOHNFKENNEDY DR,-122.457508,37.771133,2014-02-08 12:00:00,1,12,2,...,0,0,0,0,1,0,0,0,0,16
505714,SUSPICIOUS OCC,SUSPICIOUS ACT TOWARDS CHILD,1,1200 Block of PAGE ST,-122.441394,37.771727,2008-03-31 17:00:00,1,17,3,...,0,0,0,0,1,0,0,0,0,32
853258,VEHICLE THEFT,STOLEN AUTOMOBILE,4,CALIFORNIA ST / 19TH AV,-122.478580,37.784173,2003-05-01 10:00:00,1,10,5,...,0,0,0,0,0,1,0,0,0,36
679675,VEHICLE THEFT,"VEHICLE, RECOVERED, OTHER VEHICLE",5,300 Block of GATES ST,-122.414034,37.738638,2005-09-23 21:00:00,0,21,9,...,0,1,0,0,0,0,0,0,0,36
640072,ASSAULT,INFLICT INJURY ON COHABITEE,1,3200 Block of BALBOA ST,-122.493588,37.775941,2006-04-17 18:00:00,1,18,4,...,0,0,0,0,0,1,0,0,0,1


In [48]:
le.fit_transform(train_df.Category)

array([37, 21, 21, ..., 16, 35, 12])

In [49]:
train_df.shape

(875430, 22)

In [50]:
train_df['InIntersection'] = 1
train_df.loc[train_df.Address.str.contains('Block'), 'InIntersection'] = 0

test_df['InIntersection'] = 1
test_df.loc[test_df.Address.str.contains('Block'), 'InIntersection'] = 0

In [51]:
train_df.sample(2)

,Category,Descript,DayOfWeek,Address,X,Y,Date,IsDay,Hour,Month,...,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,CategoryInt,InIntersection
12243,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",1,OFARRELL ST / LEAVENWORTH ST,-122.414620,37.785583,2015-03-16 22:00:00,0,22,3,...,0,0,0,0,0,0,0,1,35,1
402416,FRAUD,FRAUDULENT USE OF AUTOMATED TELLER CARD,4,500 Block of MARKET ST,-122.399911,37.790560,2009-09-17 00:01:00,0,0,9,...,0,0,0,0,0,1,0,0,13,0


In [52]:
train_df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'Address', 'X', 'Y', 'Date',
       'IsDay', 'Hour', 'Month', 'Year', 'PdDistrict_BAYVIEW',
       'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE', 'PdDistrict_MISSION',
       'PdDistrict_NORTHERN', 'PdDistrict_PARK', 'PdDistrict_RICHMOND',
       'PdDistrict_SOUTHERN', 'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN',
       'CategoryInt', 'InIntersection'],
      dtype='object')

In [53]:
feature_cols = ['X', 'Y', 'IsDay', 'DayOfWeek', 'Month', 'Hour', 'Year', 'InIntersection',
                'PdDistrict_BAYVIEW', 'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE',
                'PdDistrict_MISSION', 'PdDistrict_NORTHERN', 'PdDistrict_PARK',
                'PdDistrict_RICHMOND', 'PdDistrict_SOUTHERN', 'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN']
target_col = 'CategoryInt'

train_x = train_df[feature_cols]
train_y = train_df[target_col]
test_x = test_df[feature_cols]

In [54]:
train_df

,Category,Descript,DayOfWeek,Address,X,Y,Date,IsDay,Hour,Month,...,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,CategoryInt,InIntersection
0,WARRANTS,WARRANT ARREST,3,OAK ST / LAGUNA ST,-122.425892,37.774599,2015-05-13 23:53:00,0,23,5,...,0,0,1,0,0,0,0,0,37,1
1,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,3,OAK ST / LAGUNA ST,-122.425892,37.774599,2015-05-13 23:53:00,0,23,5,...,0,0,1,0,0,0,0,0,21,1
2,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,3,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015-05-13 23:33:00,0,23,5,...,0,0,1,0,0,0,0,0,21,1
3,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,3,1500 Block of LOMBARD ST,-122.426995,37.800873,2015-05-13 23:30:00,0,23,5,...,0,0,1,0,0,0,0,0,16,0
4,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,3,100 Block of BRODERICK ST,-122.438738,37.771541,2015-05-13 23:30:00,0,23,5,...,0,0,0,1,0,0,0,0,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,ROBBERY ON THE STREET WITH A GUN,1,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003-01-06 00:15:00,0,0,1,...,0,0,0,0,0,0,1,0,25,1
878045,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,1,600 Block of EDNA ST,-122.447364,37.731948,2003-01-06 00:01:00,0,0,1,...,1,0,0,0,0,0,0,0,16,0
878046,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,1,5TH ST / FOLSOM ST,-122.403390,37.780266,2003-01-06 00:01:00,0,0,1,...,0,0,0,0,0,1,0,0,16,1
878047,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",1,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003-01-06 00:01:00,0,0,1,...,0,0,0,0,0,1,0,0,35,1


In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, f_train, f_test = train_test_split(train_x, train_y, test_size=0.3)

In [56]:
print(X_train.shape)
print(f_train.shape)

(612801, 18)
(612801,)


In [57]:
from xgboost import XGBClassifier
model = XGBClassifier()               
model.fit(X_train,f_train)            
y_pred = model.predict(X_test)        

In [58]:
y_pred_proba=model.predict_proba(X_test)
from sklearn.metrics import log_loss
score = log_loss(f_test, y_pred_proba)
print("LogLoss:",score)

LogLoss: 2.4407912061908688


In [59]:
from sklearn.metrics import accuracy_score   
accuracy = accuracy_score(f_test,y_pred)
print("accuarcy: %.2f%%" % (accuracy*100.0))

accuarcy: 26.16%
